'''
To do:

load dicom data

convert to nifti

run totalsegmentator - pleural effusion

save segmentation

qualitative examination in slicer

get sørensen-dice score between GT and totalSeg 
'''

In [1]:

from preprocess_tools import *
import os
import glob
import SimpleITK as sitk
from tqdm import tqdm
from scipy.spatial import distance



dataset_path = '/Lungeholdet2024/PleuraEffusion_pilot/Dataset/manifest-1603198545583/NSCLC-Radiomics'

def convertDCM(DCM_file_name: str, output_path):
    reader = sitk.ImageSeriesReader()
    dicom_names = reader.GetGDCMSeriesFileNames(DCM_file_name)
    reader.SetFileNames(dicom_names)
    image = reader.Execute()

    # Added a call to PermuteAxes to change the axes of the data
    image = sitk.PermuteAxes(image, [2, 1, 0])

    sitk.WriteImage(image, f'{output_path}.nii.gz')


c:\Users\PC\anaconda3\envs\Lungeholdet\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
files = glob.glob(os.path.join(dataset_path, "*1-0[0-4][0-9]*", "**", "**", "[0-9]-[0-9][0-9]*.dcm"))

files = [f.replace('\\', '/') for f in files]
folder_set = set()
for folder in files:
    fold = folder.split('/1-')[0]
    folder_set.add(fold)
len(folder_set)

49

In [39]:
for folder in tqdm(folder_set):
    convertDCM(folder,output_path=f'/Lungeholdet2024/PleuraEffusion_pilot/Dataset/nifti_data/{folder[85:94]}')


  4%|▍         | 2/49 [00:07<03:00,  3.83s/it]


KeyboardInterrupt: 

In [70]:
for folder in tqdm(folder_set):
    get_segmentations(f'/Lungeholdet2024/PleuraEffusion_pilot/Dataset/nifti_data/{folder[85:94]}.nii.gz',
                      f'/Lungeholdet2024/PleuraEffusion_pilot/Dataset/total_pleural_segmentation/{folder[85:94]}.nii.gz',
                      task='pleural_pericard_effusion', fast=False)

  0%|          | 0/49 [00:00<?, ?it/s]c:\Users\PC\anaconda3\envs\Lungeholdet\lib\site-packages\nnunetv2\inference\predict_from_raw_data.py:84: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experime

In [3]:
effusion_gt_path = '\Lungeholdet2024\PleuraEffusion_pilot\Effusions'
files = glob.glob(os.path.join(effusion_gt_path, "*1-0[0-4][0-9]*","*first_reviewer.*"))

files = [f.replace('\\', '/') for f in files]
gt_effusion_set = set()
for folder in files:
    fold = folder.split('/1-')[0]
    gt_effusion_set.add(fold)
gt_effusion_set

{'/Lungeholdet2024/PleuraEffusion_pilot/Effusions/LUNG1-001/LUNG1-001_effusion_first_reviewer.nii.gz',
 '/Lungeholdet2024/PleuraEffusion_pilot/Effusions/LUNG1-002/LUNG1-002_effusion_first_reviewer.nii.gz',
 '/Lungeholdet2024/PleuraEffusion_pilot/Effusions/LUNG1-005/LUNG1-005_effusion_first_reviewer.nii.gz',
 '/Lungeholdet2024/PleuraEffusion_pilot/Effusions/LUNG1-008/LUNG1-008_effusion_first_reviewer.nii.gz',
 '/Lungeholdet2024/PleuraEffusion_pilot/Effusions/LUNG1-013/LUNG1-013_effusion_first_reviewer.nii.gz',
 '/Lungeholdet2024/PleuraEffusion_pilot/Effusions/LUNG1-016/LUNG1-016_effusion_first_reviewer.nii.gz',
 '/Lungeholdet2024/PleuraEffusion_pilot/Effusions/LUNG1-018/LUNG1-018_effusion_first_reviewer.nii.gz',
 '/Lungeholdet2024/PleuraEffusion_pilot/Effusions/LUNG1-024/LUNG1-024_effusion_first_reviewer.nii.gz',
 '/Lungeholdet2024/PleuraEffusion_pilot/Effusions/LUNG1-026/LUNG1-026_effusion_first_reviewer.nii.gz',
 '/Lungeholdet2024/PleuraEffusion_pilot/Effusions/LUNG1-028/LUNG1-028_eff

In [187]:
segmentations = []
for lung_patient in tqdm(gt_effusion_set):
    totalseg = load_nifti_convert_to_numpy(f'/Lungeholdet2024/PleuraEffusion_pilot/Dataset/total_pleural_segmentation/{lung_patient[48:57]}'+'.nii.gz')
    gt_seg = load_nifti_convert_to_numpy(lung_patient)
    gt_seg=gt_seg.transpose(1,0,2)[::-1,:,:]
    totalseg=totalseg.transpose(1,2,0)
    segmentations.append((totalseg,gt_seg))


100%|██████████| 14/14 [01:51<00:00,  7.94s/it]


In [188]:
for idx, (totalseg, gt_seg) in enumerate(segmentations):
    print(totalseg.shape, gt_seg.shape)
    

(1000, 1000, 684) (1000, 1000, 684)
(1000, 1000, 612) (1000, 1000, 612)
(1000, 1000, 564) (1000, 1000, 564)
(1000, 1000, 534) (1000, 1000, 534)
(1000, 1000, 492) (1000, 1000, 492)
(1000, 1000, 666) (1000, 1000, 666)
(1000, 1000, 804) (1000, 1000, 804)
(1000, 1000, 570) (1000, 1000, 570)
(1000, 1000, 660) (1000, 1000, 660)
(1000, 1000, 594) (1000, 1000, 594)
(1000, 1000, 684) (1000, 1000, 684)
(1000, 1000, 1056) (1000, 1000, 1056)
(1000, 1000, 546) (1000, 1000, 546)
(1000, 1000, 804) (1000, 1000, 804)


In [189]:
dice_coefficients = []
def dice_coef(y_true, y_pred):
    intersect = np.multiply(np.where(y_pred==2,1,0),y_true)
    cX = np.where(y_pred==2,1,0).sum()
    cY = np.where(y_true==1,1,0).sum()
    dice = 2 * np.sum(intersect) / (cX+cY)
    return dice


for patient, lung_patient in tqdm(zip(range(len(segmentations)),gt_effusion_set)):
    im1 = np.asarray(segmentations[patient][0])
    im2 = np.asarray(segmentations[patient][1])
    
    if im1.shape != im2.shape:
        raise ValueError("Shape mismatch: im1 and im2 must have the same shape.")

    dice_coeff = dice_coef(im2,im1)
    dice_coefficients.append([lung_patient[48:57],dice_coeff])
    print(f'The dice coefficient of patient {lung_patient[55:57]} is {dice_coeff}')

1it [00:15, 15.63s/it]

The dice coefficient of patient 18 is 0.33585631790568216


2it [00:29, 14.32s/it]

The dice coefficient of patient 28 is 0.3758573519882887


3it [00:41, 13.34s/it]

...

The dice coefficient of patient 38 is 0.0009617848884588693


13it [03:22, 15.40s/it]

The dice coefficient of patient 05 is 0.0843217322546162


14it [03:39, 15.71s/it]

The dice coefficient of patient 01 is 0.494518243290598


In [183]:
# Creating the dice calculations
intersect = np.multiply(np.where(totalseg==2,1,0),gt_seg)
cX = np.where(totalseg==2,1,0).sum()
cY = np.where(gt_seg==1,1,0).sum()
dice = 2 * np.sum(intersect) / (cX+cY)